<a href="https://colab.research.google.com/github/noodlepopllc/LearnVietnamese/blob/main/Colab/Text2Speech2Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [3]:
model_name = "VietAI/envit5-translation"
#model_name = "NlpHUST/t5-en-vi-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
inputs = ["en: I do not think this can handle multiple sentences."]
outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to(device), max_length=512)
output = [output for output in tokenizer.batch_decode(outputs, skip_special_tokens=True)][0].split('vi: ')[1]
print(output)

Tôi không nghĩ nó có thể giải quyết được nhiều câu.


In [5]:
outputs = model.generate(tokenizer([f'vi: {output}'], return_tensors="pt", padding=True).input_ids.to(device), max_length=512)
test = [output for output in tokenizer.batch_decode(outputs, skip_special_tokens=True)][0].split('en: ')[1]
print(test)

I don't think it solves many of the questions.


In [6]:

import gc

model.cpu()
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()


In [7]:
from transformers import VitsModel
import scipy
from IPython.display import Audio, display

In [8]:
model = VitsModel.from_pretrained("facebook/mms-tts-vie")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-vie")
inputs = tokenizer(output, return_tensors="pt")
with torch.no_grad():
    out = model(**inputs).waveform
    sample = out.numpy()[0]
    rate = model.config.sampling_rate
    scipy.io.wavfile.write("out.wav", rate=rate, data=sample)
    audio = Audio(data=sample, rate = rate, autoplay=False)
    display(audio)

In [9]:
model.cpu()
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

In [10]:
from transformers import pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")
output = transcriber("out.wav")['text']
print(output)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cpu
`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


 Tôi không nghĩ nó có thể giải quyết được nhiều câu.


In [11]:
del transcriber
gc.collect()
torch.cuda.empty_cache()

In [12]:
model_name = "VietAI/envit5-translation"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)
inputs = [f"vi: {output}"]
print(inputs)
outputs = model.generate(tokenizer(inputs, return_tensors="pt", padding=True).input_ids.to(device), max_length=512)
output = [output for output in tokenizer.batch_decode(outputs, skip_special_tokens=True)][0].split('en: ')[1]
print(output)

['vi:  Tôi không nghĩ nó có thể giải quyết được nhiều câu.']
I don't think it solves many of the questions.
